# Задание по программированию: 1NN против RandomForest

В этом задании будет использоваться датасет digits из sklearn.datasets. Оставьте последние 25% объектов для контроля качества, разделив X и y на X_train, y_train и X_test, y_test.

Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами 1NN с RandomForestClassifier из sklearn на 1000 деревьях.

In [5]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np

In [4]:
# Загружаем матрицу признаков и ответы digits, делим на обучение и тест без перемешивания
X, y = datasets.load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, shuffle = False, random_state = 0)

## Задание 1

Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. Можно не извлекать корень из суммы квадратов отклонений, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.

Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием).

Сортировка массива длиной N требует порядка N log N сравнений (строже говоря, она работает за O(N log N)). Подумайте, как можно легко улучшить получившееся время работы. Кроме простого способа найти ближайший объект всего за N сравнений, можно попробовать придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки. За выбор метода поиска ближайших соседей в KNeighborsClassifier из sklearn отвечает параметр algorithm — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных, вам может быть интересно познакомиться со структурами данных ball tree и kd tree.

Доля ошибок, допускаемых 1NN на тестовой выборке, — ответ в задании 1.

In [14]:
# Мы имеем:
print(f'{X_train.shape[0]} точек в обучающей выборке и {X_test.shape[0]} точек в тестовой выборке')
# Нужно составить вектор предсказаний y_pred для каждой точки из тестовой выборки
# Внешний цикл - перебираем точки из тестовой выборки
# Внутренний цикл - перебираем точки из обучающей выборки
# Считаем евклидову норму разницы векторов - вектора из тестовой выборки и вектора из обучающей выборки
# Запоминаем вычисленную норму и номер строки этого вектора, считаем следующую. 
# Если следующая норма меньше предыдущей, перезапоминаем ее и ее номер.
# По завершении цикла находим по номеру наименьшей нормы, какой класс записан по этому номеру в y_train.
# Добавляем класс в y_pred.
# Переходим к следующей иттерации внешнего цикла

num = 0
norma = 1000
y_1NN_pred = []

for i in X_test:
    for ind, j in enumerate(X_train):
        spam = np.linalg.norm(i-j, ord = 2)
        if spam < norma:
            norma = spam
            num = ind
    y_1NN_pred.append(y_train[num])
    num = 0
    norma = 1000

print(len(y_1NN_pred))

1347 точек в обучающей выборке и 450 точек в тестовой выборке
450


In [19]:
# Посчитаем долю ошибок
y_1NN_pred_vec = np.array(y_1NN_pred)
score_1NN = 1 - accuracy_score(y_test, y_1NN_pred_vec)
score_1NN

0.0377777777777778

In [24]:
# Запишем 1 ответ в файл
with open('answer1.txt', 'w') as f:
    f.write(str(score_1NN))

## Задание 2

Теперь обучите на обучающей выборке RandomForestClassifier(n_estimators=1000) из sklearn. Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней. Эта доля — ответ в задании 2. Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, пожалуй, одного из самых простых методов — 1NN. Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.

In [21]:
rfc = RandomForestClassifier(n_estimators = 1000, random_state=0)
rfc.fit(X_train, y_train)
y_pred_rfc = rfc.predict(X_test)
score_rfc = 1 - accuracy_score(y_test, y_pred_rfc)
score_rfc

0.06444444444444442

In [23]:
# Запишем 2 ответ в файл
with open('answer2.txt', 'w') as f:
    f.write(str(score_rfc))